In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates

R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`



R version 3.6.0 (2019-04-26) -- "Planting of a Tree"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: This version of Shiny is designed to work with 'htmlwidgets' >= 1.5.
│     Please upgrade via install.packages('htmlwidgets').
│ Registered S3 method overwritten by 'xts':
│   method     from
│   as.zoo.xts zoo 
│ [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: -- Attaching packages --------------------------------------- tidyverse 1.2.1 --
│ v ggplot2 3.2.0     v readr   1.3.1
│ v tibble  3.0.0     v purrr   0.3.3
│ v tidyr   1.0.2     v stringr 1.4.0
│ v ggplot2 3.2.0     v forcats 0.4.0
│ Warning: package 'tibble' was built 

In [2]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
PSE_Difference = 0.00
JND_Difference = 0.00
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()
formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference |ID) + (Difference|StandardValues));
formulaNull_Interaction = @formula(Answer ~ Difference + ConditionOfInterest + 
                                            (Difference | ID) + 
                                            (Difference | StandardValues));

for reps in Range_reps
    for n in Range_Participants
        CurrentRunthrough = CurrentRunthrough + 1

        if CurrentRunthrough > 1
        print("This runthrough took ", Dates.now()-TimeStartRunthrough, " s")                
        end
        
        print("Starting runthrough N° ", CurrentRunthrough, " out of ", TotalNumber)
        global TimeStartRunthrough = Dates.now()

        for j in 1:nIterations
            
            @rput reps n j ConditionOfInterest StandardValues PSE_Difference JND_Difference Multiplicator_PSE_Standard
            @rput Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between
            
            R"""
            ID = paste0("s",1:n)
            
            TimeStartTrial = Sys.time() #get time at beginning of trial            
            Psychometric = SimulatePsychometricFunction_Staircase(ID,
                            ConditionOfInterest,
                            StandardValues,
                            1:reps,
                            PSE_Difference,
                            JND_Difference,
                            Multiplicator_PSE_Standard,
                            Multiplicator_SD_Standard,
                            SD_ResponseFunction,
                            Mean_Variability_Between,
                            SD_Variability_Between)
            
              Psychometric$StandardValues = as.character(Psychometric$StandardValues)
              Duration_SimulateDataset = Sys.time() - TimeStartTrial #get duration of fitting
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0)
              RDuration_NelderMead_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ0 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1)
              RDuration_NelderMead_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ1 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(14,16)] 
              RCoefficients_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ0 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ1 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]            
              RAIC_nloptwrap_nAGQ0 = AIC(GLMM)
            
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMMNull = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest + Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RPvalues_nloptwrap_nAGQ0_LRT = anova(GLMM,GLMMNull)$`Pr(>Chisq)`[2]
              RDuration_nloptwrap_nAGQ0_LRT = Sys.time() - TimeStartTrial #get duration of fitting            

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]            
              RAIC_nloptwrap_nAGQ1 = AIC(GLMM)
            
            
            R = data.frame(iteration = j, 
                 reps = reps,
                 n = n, 
            
                label = c("NelderMead_nAGQ0","NelderMead_nAGQ1","Bobyqa_nAGQ0", "Bobyqa_nAGQ1",
                        "nloptwrap_nAGQ0","nloptwrap_nAGQ1","R: LRT"), 
                AIC = c(RAIC_NelderMead_nAGQ0,RAIC_NelderMead_nAGQ1,RAIC_Bobyqa_nAGQ0,RAIC_Bobyqa_nAGQ1,
                        RAIC_nloptwrap_nAGQ0,RAIC_nloptwrap_nAGQ1,0),
                Pvalues_Accuracy = c(RPvalues_NelderMead_nAGQ0[1],RPvalues_NelderMead_nAGQ1[1],RPvalues_Bobyqa_nAGQ0[1],
                        RPvalues_Bobyqa_nAGQ1[1],RPvalues_nloptwrap_nAGQ0[1],RPvalues_nloptwrap_nAGQ1[1],
                        0),
                Pvalues_Interaction = c(RPvalues_NelderMead_nAGQ0[2],RPvalues_NelderMead_nAGQ1[2],RPvalues_Bobyqa_nAGQ0[2],
                        RPvalues_Bobyqa_nAGQ1[2],RPvalues_nloptwrap_nAGQ0[2],RPvalues_nloptwrap_nAGQ1[2],
                        RPvalues_nloptwrap_nAGQ0_LRT),
                Duration = c(RDuration_NelderMead_nAGQ0,RDuration_NelderMead_nAGQ1,RDuration_Bobyqa_nAGQ0,
                        RDuration_Bobyqa_nAGQ1,RDuration_nloptwrap_nAGQ0,RDuration_nloptwrap_nAGQ1,
                        RDuration_nloptwrap_nAGQ0_LRT),
                Coefficients_Accuracy = c(RCoefficients_NelderMead_nAGQ0[1],RCoefficients_NelderMead_nAGQ1[1],
                                          RCoefficients_Bobyqa_nAGQ0[1], RCoefficients_Bobyqa_nAGQ1[1],
                                          RCoefficients_nloptwrap_nAGQ0[1],RCoefficients_nloptwrap_nAGQ1[1],0),
                Coefficients_Interaction = c(RCoefficients_NelderMead_nAGQ0[2],RCoefficients_NelderMead_nAGQ1[2],
                                          RCoefficients_Bobyqa_nAGQ0[2], RCoefficients_Bobyqa_nAGQ1[2],
                                          RCoefficients_nloptwrap_nAGQ0[2],RCoefficients_nloptwrap_nAGQ1[2],0),
            
                ConditionsOfInterest=rep(length(ConditionOfInterest),7), 
                StandardValue1=rep(StandardValues[1],7),
                StandardValue2=rep(StandardValues[2],7), 
                PSE_Difference=rep(PSE_Difference,7), 
                JND_Difference=rep(JND_Difference,7), 
                Multiplicator_PSE_Standard=rep(Multiplicator_PSE_Standard,7), 
                Multiplicator_SD_Standard=rep(Multiplicator_SD_Standard,7),
                SD_ResponseFunction = rep(SD_ResponseFunction,7),
                Mean_Variability_Between = rep(Mean_Variability_Between,7),
                SD_Variability_Between = rep(SD_Variability_Between,7),
                Program = rep("R",7),
                Duration_SimulateDataset=rep(Duration_SimulateDataset,7))

            """
            
            @rget Psychometric Duration_SimulateDataset R
            
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=true)
            JuliaDurationGLMM_NeldMeader_AGP0 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_NeldMeader_AGP0 = aic(GLMM)
            JuliaPvalues_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]
          
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=false)
            JuliaDurationGLMM_NeldMeader_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_NeldMeader_AGP1 = aic(GLMM)
            JuliaPvalues_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]  

            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=false) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP1 = aic(GLMM)
            JuliaPvalues_bobyqa_AGP1 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP1 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]
            
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=true) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP0 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP0 = aic(GLMM)
            JuliaPvalues_bobyqa_AGP0 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP0 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]

            TimeStartTrial = Dates.now()
            GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull_Interaction, Psychometric, Bernoulli()),fast=true)
            Pvalue_LRT = MixedModels.likelihoodratiotest(GLMM,GLMMNull).pvalues[1]
            JuliaDurationLRT = ((Dates.now()) - TimeStartTrial)
            
            
            Julia = DataFrame(
                        iteration = j,
                        reps = reps,
                        n = n,
                        
                        label = ["JuliaAIC_NeldMeader_AGP0","JuliaAIC_bobyqa_AGP0",
                                 "JuliaAIC_NeldMeader_AGP1","JuliaAIC_bobyqa_AGP1","JuliaLRT"],
                        AIC = [JuliaAIC_NeldMeader_AGP0,JuliaAIC_bobyqa_AGP0,
                               JuliaAIC_NeldMeader_AGP1,JuliaAIC_bobyqa_AGP1,0],
                        Pvalues_Accuracy = [JuliaPvalues_NeldMeader_AGP0[1],JuliaPvalues_bobyqa_AGP0[1],
                                            JuliaPvalues_NeldMeader_AGP1[1],JuliaPvalues_bobyqa_AGP1[1],0],
                        Pvalues_Interaction = [JuliaPvalues_NeldMeader_AGP0[2],JuliaPvalues_bobyqa_AGP0[2],JuliaPvalues_NeldMeader_AGP1[2],
                                               JuliaPvalues_bobyqa_AGP1[2],Pvalue_LRT],
                        Duration = [JuliaDurationGLMM_NeldMeader_AGP0.value/1000,JuliaDurationGLMM_bobyqa_AGP0.value/1000,
                                    JuliaDurationGLMM_NeldMeader_AGP1.value/1000,JuliaDurationGLMM_bobyqa_AGP1.value/1000,
                                    JuliaDurationLRT.value/1000],
                        Coefficients_Accuracy = [JuliaCoefficients_NeldMeader_AGP0[1],JuliaCoefficients_NeldMeader_AGP1[1],
                                                 JuliaCoefficients_bobyqa_AGP1[1],JuliaCoefficients_bobyqa_AGP0[1],0],
                        Coefficients_Interaction = [JuliaCoefficients_NeldMeader_AGP0[2],JuliaCoefficients_NeldMeader_AGP1[2],
                                                 JuliaCoefficients_bobyqa_AGP1[2],JuliaCoefficients_bobyqa_AGP0[2],0],
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between,
                        Program = "Julia",
                        Duration_SimulateDataset=Duration_SimulateDataset)
            
            if CurrentRunthrough == 1 && j == 1 && reps == Range_reps[1] && n == Range_Participants[1]
            
                global PowerfulDataframe = append!(R,Julia)
                
            else
                
                PowerfulDataframe = append!(PowerfulDataframe,R)
                PowerfulDataframe = append!(PowerfulDataframe,Julia)
            end
        end
    end
end

CSV.write(join(["Pvalues_JuliaSmallerModelNoEffect.csv"]),PowerfulDataframe)

Starting runthrough N° 1 out of 24

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.203547 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RC

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00538386 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│ 

This runthrough took 358357 milliseconds sStarting runthrough N° 2 out of 

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.0112866 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


...

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.006305 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RC

This runthrough took 1663868 milliseconds sStarting runthrough N° 24 out of 24

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\pac

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   unable to evaluate scaled gradient
│ Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge: degenerate  Hessian with 1 negative eigenvalues
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\sr

"Pvalues_JuliaSmallerModelNoEffect.csv"

In [3]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
PSE_Difference = 0.025
JND_Difference = 0.05
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()
formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference |ID) + (Difference|StandardValues));
formulaNull_Interaction = @formula(Answer ~ Difference + ConditionOfInterest + 
                                            (Difference | ID) + 
                                            (Difference | StandardValues));

for reps in Range_reps
    for n in Range_Participants
        CurrentRunthrough = CurrentRunthrough + 1

        if CurrentRunthrough > 1
        print("This runthrough took ", Dates.now()-TimeStartRunthrough, " s")                
        end
        
        print("Starting runthrough N° ", CurrentRunthrough, " out of ", TotalNumber)
        global TimeStartRunthrough = Dates.now()

        for j in 1:nIterations
            
            @rput reps n j ConditionOfInterest StandardValues PSE_Difference JND_Difference Multiplicator_PSE_Standard
            @rput Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between
            
            R"""
            ID = paste0("s",1:n)
            
            TimeStartTrial = Sys.time() #get time at beginning of trial            
            Psychometric = SimulatePsychometricFunction_Staircase(ID,
                            ConditionOfInterest,
                            StandardValues,
                            1:reps,
                            PSE_Difference,
                            JND_Difference,
                            Multiplicator_PSE_Standard,
                            Multiplicator_SD_Standard,
                            SD_ResponseFunction,
                            Mean_Variability_Between,
                            SD_Variability_Between)
            
              Psychometric$StandardValues = as.character(Psychometric$StandardValues)
              Duration_SimulateDataset = Sys.time() - TimeStartTrial #get duration of fitting
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0)
              RDuration_NelderMead_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ0 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1)
              RDuration_NelderMead_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ1 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(14,16)] 
              RCoefficients_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ0 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ1 = AIC(GLMM)

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]            
              RAIC_nloptwrap_nAGQ0 = AIC(GLMM)
            
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMMNull = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest + Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RPvalues_nloptwrap_nAGQ0_LRT = anova(GLMM,GLMMNull)$`Pr(>Chisq)`[2]
              RDuration_nloptwrap_nAGQ0_LRT = Sys.time() - TimeStartTrial #get duration of fitting            

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]            
              RAIC_nloptwrap_nAGQ1 = AIC(GLMM)
            
            
            R = data.frame(iteration = j, 
                 reps = reps,
                 n = n, 
            
                label = c("NelderMead_nAGQ0","NelderMead_nAGQ1","Bobyqa_nAGQ0", "Bobyqa_nAGQ1",
                        "nloptwrap_nAGQ0","nloptwrap_nAGQ1","R: LRT"), 
                AIC = c(RAIC_NelderMead_nAGQ0,RAIC_NelderMead_nAGQ1,RAIC_Bobyqa_nAGQ0,RAIC_Bobyqa_nAGQ1,
                        RAIC_nloptwrap_nAGQ0,RAIC_nloptwrap_nAGQ1,0),
                Pvalues_Accuracy = c(RPvalues_NelderMead_nAGQ0[1],RPvalues_NelderMead_nAGQ1[1],RPvalues_Bobyqa_nAGQ0[1],
                        RPvalues_Bobyqa_nAGQ1[1],RPvalues_nloptwrap_nAGQ0[1],RPvalues_nloptwrap_nAGQ1[1],
                        0),
                Pvalues_Interaction = c(RPvalues_NelderMead_nAGQ0[2],RPvalues_NelderMead_nAGQ1[2],RPvalues_Bobyqa_nAGQ0[2],
                        RPvalues_Bobyqa_nAGQ1[2],RPvalues_nloptwrap_nAGQ0[2],RPvalues_nloptwrap_nAGQ1[2],
                        RPvalues_nloptwrap_nAGQ0_LRT),
                Duration = c(RDuration_NelderMead_nAGQ0,RDuration_NelderMead_nAGQ1,RDuration_Bobyqa_nAGQ0,
                        RDuration_Bobyqa_nAGQ1,RDuration_nloptwrap_nAGQ0,RDuration_nloptwrap_nAGQ1,
                        RDuration_nloptwrap_nAGQ0_LRT),
                Coefficients_Accuracy = c(RCoefficients_NelderMead_nAGQ0[1],RCoefficients_NelderMead_nAGQ1[1],
                                          RCoefficients_Bobyqa_nAGQ0[1], RCoefficients_Bobyqa_nAGQ1[1],
                                          RCoefficients_nloptwrap_nAGQ0[1],RCoefficients_nloptwrap_nAGQ1[1],0),
                Coefficients_Interaction = c(RCoefficients_NelderMead_nAGQ0[2],RCoefficients_NelderMead_nAGQ1[2],
                                          RCoefficients_Bobyqa_nAGQ0[2], RCoefficients_Bobyqa_nAGQ1[2],
                                          RCoefficients_nloptwrap_nAGQ0[2],RCoefficients_nloptwrap_nAGQ1[2],0),
            
                ConditionsOfInterest=rep(length(ConditionOfInterest),7), 
                StandardValue1=rep(StandardValues[1],7),
                StandardValue2=rep(StandardValues[2],7), 
                PSE_Difference=rep(PSE_Difference,7), 
                JND_Difference=rep(JND_Difference,7), 
                Multiplicator_PSE_Standard=rep(Multiplicator_PSE_Standard,7), 
                Multiplicator_SD_Standard=rep(Multiplicator_SD_Standard,7),
                SD_ResponseFunction = rep(SD_ResponseFunction,7),
                Mean_Variability_Between = rep(Mean_Variability_Between,7),
                SD_Variability_Between = rep(SD_Variability_Between,7),
                Program = rep("R",7),
                Duration_SimulateDataset=rep(Duration_SimulateDataset,7))

            """
            
            @rget Psychometric Duration_SimulateDataset R
            
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=true)
            JuliaDurationGLMM_NeldMeader_AGP0 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_NeldMeader_AGP0 = aic(GLMM)
            JuliaPvalues_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]
          
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=false)
            JuliaDurationGLMM_NeldMeader_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_NeldMeader_AGP1 = aic(GLMM)
            JuliaPvalues_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]  

            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=false) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP1 = aic(GLMM)
            JuliaPvalues_bobyqa_AGP1 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP1 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]
            
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=true) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP0 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP0 = aic(GLMM)
            JuliaPvalues_bobyqa_AGP0 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP0 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]

            TimeStartTrial = Dates.now()
            GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull_Interaction, Psychometric, Bernoulli()),fast=true)
            Pvalue_LRT = MixedModels.likelihoodratiotest(GLMM,GLMMNull).pvalues[1]
            JuliaDurationLRT = ((Dates.now()) - TimeStartTrial)
            
            
            Julia = DataFrame(
                        iteration = j,
                        reps = reps,
                        n = n,
                        
                        label = ["JuliaAIC_NeldMeader_AGP0","JuliaAIC_bobyqa_AGP0",
                                 "JuliaAIC_NeldMeader_AGP1","JuliaAIC_bobyqa_AGP1","JuliaLRT"],
                        AIC = [JuliaAIC_NeldMeader_AGP0,JuliaAIC_bobyqa_AGP0,
                               JuliaAIC_NeldMeader_AGP1,JuliaAIC_bobyqa_AGP1,0],
                        Pvalues_Accuracy = [JuliaPvalues_NeldMeader_AGP0[1],JuliaPvalues_bobyqa_AGP0[1],
                                            JuliaPvalues_NeldMeader_AGP1[1],JuliaPvalues_bobyqa_AGP1[1],0],
                        Pvalues_Interaction = [JuliaPvalues_NeldMeader_AGP0[2],JuliaPvalues_bobyqa_AGP0[2],JuliaPvalues_NeldMeader_AGP1[2],
                                               JuliaPvalues_bobyqa_AGP1[2],Pvalue_LRT],
                        Duration = [JuliaDurationGLMM_NeldMeader_AGP0.value/1000,JuliaDurationGLMM_bobyqa_AGP0.value/1000,
                                    JuliaDurationGLMM_NeldMeader_AGP1.value/1000,JuliaDurationGLMM_bobyqa_AGP1.value/1000,
                                    JuliaDurationLRT.value/1000],
                        Coefficients_Accuracy = [JuliaCoefficients_NeldMeader_AGP0[1],JuliaCoefficients_NeldMeader_AGP1[1],
                                                 JuliaCoefficients_bobyqa_AGP1[1],JuliaCoefficients_bobyqa_AGP0[1],0],
                        Coefficients_Interaction = [JuliaCoefficients_NeldMeader_AGP0[2],JuliaCoefficients_NeldMeader_AGP1[2],
                                                 JuliaCoefficients_bobyqa_AGP1[2],JuliaCoefficients_bobyqa_AGP0[2],0],
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between,
                        Program = "Julia",
                        Duration_SimulateDataset=Duration_SimulateDataset)
            
            if CurrentRunthrough == 1 && j == 1 && reps == Range_reps[1] && n == Range_Participants[1]
            
                global PowerfulDataframe = append!(R,Julia)
                
            else
                
                PowerfulDataframe = append!(PowerfulDataframe,R)
                PowerfulDataframe = append!(PowerfulDataframe,Julia)
            end
        end
    end
end

CSV.write(join(["Pvalues_JuliaSmallerModelSmallEffect.csv"]),PowerfulDataframe)

Starting runthrough N° 1 out of 24

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00245057 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   unable to evaluate scaled gradient
│ Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge: degenerate  Hessian with 1 negative eigenvalues
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\sr

This runthrough took 355870 milliseconds sStarting runthrough N° 2 out of 24

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\pac

...

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.0108887 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


24

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   unable to evaluate scaled gradient
│ Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge: degenerate  Hessian with 1 negative eigenvalues
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\sr

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\pac

"Pvalues_JuliaSmallerModelSmallEffect.csv"

1601 milliseconds